# 2: Extract Contributions of Latent Features & Saliency Maps:


In [ ]:
import pandas as pd
import numpy as np
import pickle
import time
import lime
import lime.lime_tabular
# import deeplift

from collections import Counter

from copy import deepcopy

from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import backend as K

# For Deep Learning Explanations
# from deepexplain.tensorflow import DeepExplain
# from deeplift.conversion import kerasapi_conversion as kc

In [ ]:
# Load Data
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

oh_y_train = np.load("oh_y_train.npy")
oh_y_test = np.load("oh_y_test.npy")

img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)


In [ ]:
#### NB: MAKE SURE CORRECT MODEL
model = load_model("NN2.h5")

In [ ]:
# For indexing
nn_preds_test = model.predict(X_test)
nn_preds_train = model.predict(X_train)

## Get activations

In [ ]:
def get_activations(model, model_inputs, print_shape_only=False, layer_name=None):

    activations = []
    inp = model.input

    model_multi_inputs_cond = True
    if not isinstance(inp, list):
        # only one input! let's wrap it in a list.
        inp = [inp]
        model_multi_inputs_cond = False

    outputs = [layer.output for layer in model.layers if
               layer.name == layer_name or layer_name is None]  # all layer outputs

    funcs = [K.function(inp + [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

    if model_multi_inputs_cond:
        list_inputs = []
        list_inputs.extend(model_inputs)
        list_inputs.append(0.)
    else:
        list_inputs = [model_inputs, 0.]

    # Learning phase. 0 = Test mode (no dropout or batch normalization)
    # layer_outputs = [func([model_inputs, 0.])[0] for func in funcs]
    layer_outputs = [func(list_inputs)[0] for func in funcs]
    for layer_activations in layer_outputs:
        activations.append(layer_activations)

        return activations


X_train_act = list()
X_test_act = list()

for i in range(0, 60000, 1000):
    #print(i)
    start = i
    end = i + 1000
    
    X_train_act_seg = get_activations(model, X_train[start: end])
    X_train_act.append(X_train_act_seg)
    
    
for i in range(0, 10000, 1000):
    #print(i)
    start = i
    end = i + 1000
    
    X_test_act_seg = get_activations(model, X_test[start: end])
    X_test_act.append(X_test_act_seg)

In [ ]:
X_train_act_new = list()
X_test_act_new = list()

for group in X_train_act:
    for i in range(len(group)):
        X_train_act_new.append(group[i])
                
for group in X_test_act:
    for i in range(len(group)):
        X_test_act_new.append(group[i])        

X_train_act = np.array(X_train_act_new)
X_test_act = np.array(X_test_act_new)

np.save("X_train_act", X_train_act)
np.save("X_test_act", X_test_act)

## Get DeepLIFT contributions

In [ ]:
from copy import deepcopy

deeplift_model =\
    kc.convert_model_from_saved_files(
        "NN.h5",
        nonlinear_mxts_mode=deeplift.layers.NonlinearMxtsMode.DeepLIFT_GenomicsDefault) 
    
# Which layer to propagate contribution scores?
find_scores_layer_idx = -8

deeplift_contribs_func = deeplift_model.get_target_contribs_func(
                            find_scores_layer_idx=find_scores_layer_idx,
                            target_layer_idx=-2)


# Need to iterate scores for each output neuron one at a time (10 for MNIST)
for i in range(model.get_weights()[-1].shape[0]):
    
    train = np.array(deeplift_contribs_func(task_idx=i,
                                             input_data_list=[X_train],
                                             batch_size=10,
                                             progress_update=1000))

    test = np.array(deeplift_contribs_func(task_idx=i,
                                             input_data_list=[X_test],
                                             batch_size=10,
                                             progress_update=1000))
    
    train = np.array(train)
    test = np.array(test)
    train = np.expand_dims(train, 1)
    test = np.expand_dims(test, 1)
    
    if i == 0:
        X_train_deeplift = deepcopy(train)
        X_test_deeplift = deepcopy(test)
    else:
        X_train_deeplift = np.append(X_train_deeplift, train, axis=1)
        X_test_deeplift = np.append(X_test_deeplift, test, axis=1)

In [ ]:
train = deepcopy(X_train_deeplift)
test = deepcopy(X_test_deeplift)

In [ ]:
X_train_deeplift = list()
X_test_deeplift = list()

for i in range(len(nn_preds_train)):
    index = nn_preds_train[i]
    X_train_deeplift.append(train[i][index])

for i in range(len(nn_preds_test)):
    index = nn_preds_test[i]
    X_test_deeplift.append(test[i][index])

In [ ]:
X_train_deeplift = np.array(X_train_deeplift)
X_test_deeplift = np.array(X_test_deeplift)

In [ ]:
print("Training:", X_train_deeplift.shape)
print("Testing:", X_test_deeplift.shape)

np.save("new_X_train_deeplift", X_train_deeplift)
np.save("new_X_test_deeplift", X_test_deeplift)

In [ ]:
# Load Data
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

oh_y_train = np.load("oh_y_train.npy")
oh_y_test = np.load("oh_y_test.npy")

In [ ]:
#### NB: MAKE SURE CORRECT MODEL
model = load_model("NN.h5")

In [ ]:
# For indexing
nn_preds_test = model.predict_classes(X_test)
nn_preds_train = model.predict_classes(X_train)

## Get Activations

In [ ]:
def get_activations(model, model_inputs, print_shape_only=False, layer_name=None):

    activations = []
    inp = model.input

    model_multi_inputs_cond = True
    if not isinstance(inp, list):
        # only one input! let's wrap it in a list.
        inp = [inp]
        model_multi_inputs_cond = False

    outputs = [layer.output for layer in model.layers if
               layer.name == layer_name or layer_name is None]  # all layer outputs

    funcs = [K.function(inp + [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

    if model_multi_inputs_cond:
        list_inputs = []
        list_inputs.extend(model_inputs)
        list_inputs.append(0.)
    else:
        list_inputs = [model_inputs, 0.]

    # Learning phase. 0 = Test mode (no dropout or batch normalization)
    # layer_outputs = [func([model_inputs, 0.])[0] for func in funcs]
    layer_outputs = [func(list_inputs)[0] for func in funcs]
    for layer_activations in layer_outputs:
        activations.append(layer_activations)

    return activations

## Get Contributions for Latent Features:
Simply multiply the activations by the weights connecting to the class in question.

In [ ]:
def get_activations(model, model_inputs, print_shape_only=False, layer_name=None):

    activations = []
    inp = model.input

    model_multi_inputs_cond = True
    if not isinstance(inp, list):
        # only one input! let's wrap it in a list.
        inp = [inp]
        model_multi_inputs_cond = False

    outputs = [layer.output for layer in model.layers if
               layer.name == layer_name or layer_name is None]  # all layer outputs

    funcs = [K.function(inp + [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

    if model_multi_inputs_cond:
        list_inputs = []
        list_inputs.extend(model_inputs)
        list_inputs.append(0.)
    else:
        list_inputs = [model_inputs, 0.]

    # Learning phase. 0 = Test mode (no dropout or batch normalization)
    # layer_outputs = [func([model_inputs, 0.])[0] for func in funcs]
    layer_outputs = [func(list_inputs)[0] for func in funcs]
    for layer_activations in layer_outputs:
        activations.append(layer_activations)

    return activations


X_train_act_seg = get_activations(model, X_train)[-4]
X_test_act_seg = get_activations(model, X_test)[-4]

In [ ]:
np.save("X_train_act", X_train_act_seg)
np.save("X_test_act", X_test_act_seg)

### Get Contribution of Features to Classification

In [ ]:
nn_preds_test = model.predict_classes(X_test)
nn_preds_train = model.predict_classes(X_train)

In [ ]:
# maybe double check???
weights = model.layers[-2].get_weights()[0].T

In [ ]:
X_train_cont = deepcopy(X_train_act_seg)
X_test_cont = deepcopy(X_test_act_seg)

for i in range(len(nn_preds_train)):
    X_train_cont[i] = X_train_cont[i] * weights[nn_preds_train[i]]
    
for i in range(len(nn_preds_test)):
    X_test_cont[i] = X_test_cont[i] * weights[nn_preds_test[i]]

In [ ]:
np.save("X_train_cont", X_train_cont)
np.save("X_test_cont", X_test_cont)